# S2W7D4: 诊断与调优 (Diagnosis & Optimization)
昨天我们给模型做了一次“体检”，发现它虽然及格了（Accuracy 60%），但是有点“偏科”（对稀有样本 Grab 不敢预测）。同时我们修改了代码，引入了 **Weighted Loss** 试图矫正这个问题。

今天，我们要化身为 **“AI 医生”**。我们不能只看分数的涨跌，我们要**看着病人的眼睛**（查看错误样本），找出它到底在哪里犯糊涂，然后通过**调参**（开药）来治好它。

**这是面试中“项目深挖”环节最容易被问到的地方**：“你遇到过 Bad Case 吗？你是怎么分析的？最后怎么解决的？”

## 1 🔍 核心任务：可视化“坏样本” (Bad Case Analysis)

Loss 只是一个数字，它不会告诉你模型为什么把“向左转”听成了“停止”。我们需要把预测错误的样本打印出来。

In [1]:
import sys
sys.path.append("../../") 
import torch
import numpy as np
import pandas as pd
from transformers import Trainer, TrainingArguments, BertTokenizer
from src.config import PRETRAINED_MODEL_DIR, CHECKPOINT_DIR
from src.models.model_bert import BertClassifier
from src.dataset.dataset import IntentDataset
from src.metrics import compute_metrics

# 1. 准备验证集 (这次多造一点难的数据，模拟真实场景)
val_texts = [
    "向左转", "停止", "抓起来", "请勿移动", "前进", 
    "把那个杯子拿给我", "紧急制动", "慢慢往前走", "向右转动一点点", "松开手"
]
# 假设 0:MOVE, 1:STOP, 2:GRAB
val_labels = [0, 1, 2, 1, 0, 2, 1, 0, 0, 2]
label_map = {0: "MOVE", 1: "STOP", 2: "GRAB"}

# 2. 加载昨天训练好的模型 (Checkpoint-24)
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_DIR)
val_dataset = IntentDataset(val_texts, val_labels, tokenizer)

model = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=3)
# 修正加载方式，使用 safetensors
from safetensors.torch import load_file
# 请确认 checkpoint-24 是你昨天的最佳权重目录
best_ckpt = CHECKPOINT_DIR / "checkpoint-24" 
state_dict = load_file(f"{best_ckpt}/model.safetensors")
model.load_state_dict(state_dict)

# 3. 进行预测
training_args = TrainingArguments(output_dir="./results", per_device_eval_batch_size=4)
trainer = Trainer(model=model, args=training_args)
predictions = trainer.predict(val_dataset)

# 4. 提取结果
pred_logits = predictions.predictions
pred_ids = np.argmax(pred_logits, axis=-1)
true_ids = predictions.label_ids

# 5. ★★★ 核心：找出 Bad Cases ★★★
bad_cases = []
for i in range(len(val_texts)):
    if pred_ids[i] != true_ids[i]:
        bad_cases.append({
            "Text": val_texts[i],
            "True Label": label_map[true_ids[i]],
            "Pred Label": label_map[pred_ids[i]],
            "Confidence": np.max(torch.softmax(torch.tensor(pred_logits[i]), dim=-1).numpy())
        })

# 6. 用 Pandas 漂亮地打印出来
df = pd.DataFrame(bad_cases)
print("❌ 错误样本分析报告:")
if not df.empty:
    print(df.to_markdown())
else:
    print("🎉 太棒了！验证集全对！(说明该增加难度了)")

❌ 错误样本分析报告:
|    | Text             | True Label   | Pred Label   |   Confidence |
|---:|:-----------------|:-------------|:-------------|-------------:|
|  0 | 抓起来           | GRAB         | MOVE         |     0.474486 |
|  1 | 请勿移动         | STOP         | MOVE         |     0.420997 |
|  2 | 把那个杯子拿给我 | GRAB         | MOVE         |     0.462568 |
|  3 | 松开手           | GRAB         | MOVE         |     0.48105  |


**诊断思考 (Interview Point)**:

  * 如果模型把 "请勿移动" 预测成 `MOVE`：说明它只关注到了关键词 "移动"，没学到否定词 "请勿"。 -\> **对策**：增加带有否定词的训练数据。
  * 如果模型把 "抓起来" 预测成 `MOVE`：说明它还没学会 `GRAB` 这个生僻类别。 -\> **对策**：昨天加的 Weighted Loss 应该能缓解这个问题。

## 2 🎛️ 核心任务：启用 Weighted Loss 重训

昨天我们修改了 `model_bert.py`，但还没真正用它重新训练。今天我们要用带权重的 Loss 再跑一次，看看能不能把昨天 Precision=0 的尴尬解决掉。

### 步骤

1.  **重启 Kernel** (确保 `src.models.model_bert` 的修改生效)。
2.  **复制 Day 2 的训练代码** (或者直接在 Day 4 Notebook 里重写一遍 Trainer 流程)。
3.  **运行训练**。
4.  **再次评估**：重点看 `GRAB` 类别的 Recall 有没有上升。

**代码片段 (Day 4 Notebook):**

```python
# ... 定义 train_dataset (记得包含一些 Label=2 的样本) ...

# 重新初始化模型 (这时候它会读取我们在 init 里写好的 class_weights)
model_weighted = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=3)

trainer = Trainer(
    model=model_weighted,
    args=training_args, # 稍微调大 epoch 试试，比如 5
    train_dataset=train_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
# 训练完立刻评估
print(trainer.evaluate())
```

In [ ]:
import sys
sys.path.append("../../") 

import torch
import numpy as np
from transformers import Trainer, TrainingArguments, BertTokenizer
from src.config import PRETRAINED_MODEL_DIR, CHECKPOINT_DIR
from src.models.model_bert import BertClassifier
from src.dataset.dataset import IntentDataset
from src.metrics import compute_metrics
from src.config import CHECKPOINT_DIR
import shutil # 用于清理旧文件

# -----------------------------------------------------------------------------
# 1. 数据准备 (保持不变)
# -----------------------------------------------------------------------------
train_texts = ["向前走", "向后退", "左转", "右转", "前进十米", "后退五步", "往左边移动", "往右边移动", "一直走", "慢速前进", "全速前进", "转个圈", "停下", "紧急制动", "别动", "抓取杯子", "把苹果拿起来", "松开夹爪"]
train_labels = [0]*12 + [1]*3 + [2]*3

val_texts = ["向左转", "停止", "抓起来", "请勿移动", "前进", "把那个杯子拿给我", "松开手"]
val_labels = [0, 1, 2, 1, 0, 2, 2]

tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_DIR)
train_dataset = IntentDataset(train_texts, train_labels, tokenizer)
val_dataset = IntentDataset(val_texts, val_labels, tokenizer)

# -----------------------------------------------------------------------------
# 2. 初始化模型 (Weighted Loss)
# -----------------------------------------------------------------------------
model_weighted = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=3)
print("✅ 模型已加载 (Weighted Loss 启用)")

# -----------------------------------------------------------------------------
# 3. ★★★ 核心修改：配置保存策略 ★★★
# -----------------------------------------------------------------------------
# 定义一个专门的保存路径，方便 Day 5 调用
save_dir = str(CHECKPOINT_DIR)

training_args = TrainingArguments(
    output_dir=save_dir,             # ★ 修改：指定明确的保存路径
    num_train_epochs=15,             # 多跑几轮，确保收敛
    per_device_train_batch_size=4,
    learning_rate=2e-5,
    logging_steps=5,
    
    # --- 评估与保存策略 ---
    eval_strategy="epoch",           # 每个 epoch 评估一次
    save_strategy="epoch",           # ★ 修改：每个 epoch 保存一次权重 (必须开启！)
    save_total_limit=3,              # ★ 修改：只保留 1 个最新的，防止硬盘爆炸
    load_best_model_at_end=True,     # ★ 修改：训练结束时，自动加载效果最好的那个模型
    metric_for_best_model="f1",      # 以 F1 分数作为衡量标准
    
    no_cuda=False,
    seed=42
)

# -----------------------------------------------------------------------------
# 4. 训练与评估
# -----------------------------------------------------------------------------
trainer = Trainer(
    model=model_weighted,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print(f"🚀 开始重训并保存至 {save_dir} ...")
trainer.train()

# ★★★ 额外保险：手动保存最终的最佳模型 ★★★
final_path = f"{save_dir}/final_best"
trainer.save_model(final_path)
print(f"\n✅ 最佳模型已强制保存至: {final_path}")
print("👉 请在 Day 5 的推理代码中使用这个路径！")

✅ 模型已加载 (Weighted Loss 启用)
🚀 开始重训并保存至 /home/goodminton/study/AI-Interview-Sprint/output/train/checkpoints ...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.174000,1.178933,0.285714,0.126984,0.081633,0.285714
2,0.817700,1.140815,0.285714,0.142857,0.095238,0.285714
3,0.811100,0.925142,0.571429,0.557143,0.714286,0.571429
4,0.659900,0.930355,0.714286,0.714286,0.761905,0.714286
5,0.568900,0.946947,0.714286,0.723810,0.857143,0.714286
6,0.445700,0.867834,0.857143,0.847619,0.904762,0.857143
7,0.361900,0.759284,0.857143,0.847619,0.904762,0.857143
8,0.333200,0.739826,0.857143,0.847619,0.904762,0.857143
9,0.308700,0.734036,0.857143,0.847619,0.904762,0.857143
10,0.233600,0.714262,0.857143,0.847619,0.904762,0.857143


/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



✅ 最佳模型已强制保存至: /home/goodminton/study/AI-Interview-Sprint/output/train/checkpoints/final_best
👉 请在 Day 5 的推理代码中使用这个路径！


## 3 🧪 进阶任务：超参数调优 (Hyperparameter Tuning)

算法工程师的日常就是“炼丹”。你需要手动调整 `TrainingArguments` 里的参数，感受它们对结果的影响。

请尝试修改以下参数，并记录 Accuracy 的变化：

| 参数 | 建议尝试值 | 预期影响 |
| :--- | :--- | :--- |
| **Learning Rate** | `5e-5` (变大) | 收敛变快，但可能震荡不收敛 (Loss 乱跳)。 |
| **Learning Rate** | `1e-5` (变小) | 收敛极慢，需要更多 Epoch，但结果可能更精细。 |
| **Batch Size** | `4` 或 `8` | 梯度更稳定，但显存占用增加。 |
| **Num Epochs** | `5` 或 `10` | 训练更久，小心过拟合 (Val Loss 升高)。 |

**面试必问**: *“你在这个项目中使用了什么 Learning Rate？为什么？”*
**标准答案**: *“我使用了 BERT 微调的黄金区间 2e-5。我尝试过 5e-5，发现 Loss 震荡比较大；尝试过 1e-5，收敛太慢。2e-5 是一个比较好的平衡点。”*

## 4 ⚔️ 每日算法题 (LeetCode 8)

既然我们在处理 NLP 意图识别（把字符串变成指令），今天我们来做一道极其经典的**字符串解析 (String Parsing)** 题目。这道题考察的是你处理“脏数据”和边界条件的细心程度，这在 NLP 预处理中非常重要。

### 🎯 题目: [LeetCode 8. 字符串转换整数 (atoi)](../../LeetCode%20practice/1-50.ipynb)

  * **难度**: Medium
  * **标签**: 字符串
  * **描述**: 实现 `myAtoi(string s)` 函数，将字符串转换成一个 32 位有符号整数。
      * **步骤**:
        1.  **丢弃前导空格**。
        2.  **检查符号** (`+` 或 `-`)。
        3.  **读入数字**：直到到达非数字字符或结尾。
        4.  **截断**：如果超过 32 位整数范围 $[-2^{31}, 2^{31}-1]$，则截断。
      * **示例**: `"   -42 with words"` -\> `-42`
  * **为什么选这题**: 你的“指令识别”项目中，如果用户输入 "向左转 30 度"，你需要解析出 `30` 这个参数。这道题就是参数解析的简化版。

## ✅ Day 4 任务清单

1.  **坏样本可视化**: 运行代码，打印出 Pandas 表格，亲眼看看模型错在哪。
2.  **验证 Weighted Loss**: 重新训练模型，观察 `eval_recall` 是否提升，确认模型是否敢于预测 `GRAB` 了。
3.  **手动调参**: 至少尝试修改一次 Learning Rate，并记录结果对比。
4.  **算法**: 通过 LeetCode 8。